In [ ]:
import pandas, keras
import numpy as np
from sklearn.metrics import roc_auc_score
from IPython.display import FileLink, FileLinks
import matplotlib.pyplot as plt

In [ ]:
# These lines are not needed
import shutil
import os


cdir = os.getcwd()
original = cdir + '/output_final.h5'
target = cdir + '/output_final_copy.h5'

if os.path.exists(target):
    os.remove(target)
else:
    print("Can not delete the file as it doesn't exists")

shutil.copyfile(original,target)

In [ ]:
# 1 image has 6x6 pixels = 36 pixels 
###################### Need to try 16 x 16 and 8 x 8 and see the effect (making a heatmap for 3,4 hits), can be changed in Convertroototpand.py
pixels = ["pixel_{0}".format(i) for i in range(36)]
def to_image(df):
    return  np.expand_dims(np.expand_dims(df[pixels], axis=-1).reshape(-1,6,6), axis=-1)


# You need to email hichemb@princeton.edu to request access to this directory, or else this line will not work
pandas.read_hdf("/eos/user/h/hboucham/SWAN_projects/MergedHits/output_final.h5",mode='r')
# replace this sample every time you train CNN
#store_train = pandas.HDFStore("output_final_copy.h5")

# kinda like tree1
df = store_train.select("df",stop=-1) 

#cut on DeltaR and nUniqueSimTracksInSharedHit
##################################################### Need to move this to ConvertRoottoPanda.py
df = df[(df["GenDeltaR"]<0.1) & (df["nUniqueSimTracksInSharedHit"]>-1)]
#images_train = to_image(df_train)


# Don't need these since I'm using the same files and splitting it, might be useful if I'm using a training file and a testing file
#store_test = pandas.HDFStore("pixelTrain30k.h5") #######
#df_test = store_test.select("df",stop=-1)
#df_test = df_test[(df_test["GenDeltaR"]<0.1) & (df_test["nUniqueSimTracksInSharedHit"]>-1)]0 if shared hit but not merged, -1 if neither, 1, 2,3 if sahred and merged
#images_test = to_image(df_test)
#comb = pandas.concat([df_train,df_test])
#df_train=comb.sample(frac=0.6) ######
##########################################################################







#***************************************************** Jack's notebook: only include shared hits so 0 events are not included here !
# Make all test, train data merged hit
#df_test = df_test[(df_test["isSharedHit"]>0)]
#df_train = df_train[(df_train["isSharedHit"]>0)]

In [ ]:
#Throwing away events that have single pixel since the neural network cannot tell whether it's merged or not, there is simply not enough information.
pixelColumns = ["pixel_%i" % x for x in range(36)]
pixels_df = df[pixelColumns].values
print(pixels_df[0].shape)
pixel_number = pixels_df.astype(bool).sum(axis=1)
df.insert(0, "Pixel_number", pixel_number)
print df.info()
df = df[df["Pixel_number"]>1]
print df.info()

In [ ]:
# frac=0.5 sets half training and half testing
df_train=df.sample(frac=0.5)
df_test=df.drop(df_train.index)
images_train = to_image(df_train)
images_test = to_image(df_test)

In [ ]:
# Sanity check: printout (events, variables) Variables include 36 pixels, and a few others you can find in converroottopands.py
print df_test.shape
print df_train.shape
print df.info()

In [ ]:
import tensorflow as tf

csv_logger = tf.keras.callbacks.CSVLogger('training.txt', separator=",", append=False)

In [ ]:
# This is the CNN
# Adding layers to Neural Network: (1) is convolutional,(1.5) 2D layer ,(2) flatten output then feed it to (3) which is a regular neural network.
# (4) drops nodes in NN to avoid overfitting, finallly (5) outputs 2 values (prob(notmergedhit), prob(merged hit)), should add up to 1

from keras.layers import Dense, Dropout, Activation
# Define the network
model = keras.models.Sequential()

#layer (1)
model.add(keras.layers.Conv2D(32, kernel_size=(4,4), padding='same', activation='relu'))

#layer (1.5)
############################################## 2D layers: need to add these back individually and see their effect
#********************************************# what does this layer do again ?
#model.add(keras.layers.Conv2D(16, kernel_size=(4,4), padding='same', activation='relu'))
#model.add(keras.layers.Conv2D(16, kernel_size=(2,2), padding='same', activation='relu'))
#model.add(keras.layers.Conv2D(16, kernel_size=(1,1), padding='same', activation='relu'))

# max pooling 2D
#model.add(keras.layers.MaxPooling2D(pool_size=(2, 2), padding='same', data_format="channels_last"))

# layer (2)
model.add(keras.layers.Flatten(input_shape=(6,6,1),data_format = "channels_last"))
#model.add(keras.layers.Dense(200, activation='relu'))

# layer (3)
model.add(keras.layers.Dense(50, activation='relu'))

# layer 4, dropout 10%
model.add(Dropout(0.1))
model.add(keras.layers.Dense(2, activation='softmax'))
#print(model.summary())
                                                    
# Layer (5), train the network
###############################################################  try to find a way to include pt, eta, phi after first layer (convolutional)
# optimal number of epochs, var name might be model.loss.
model.compile(loss='categorical_crossentropy', optimizer="adam", metrics = ["accuracy"]) 
model.fit(images_train, keras.utils.to_categorical(df_train["nUniqueSimTracksInSharedHit"]>1), callbacks=[csv_logger], epochs=100, validation_split=0.1) 
# validation fraction is fraction of training sample used for testing (here 10%), epochs: number of times you run the CNN.

model.summary()



In [ ]:
# LOSS(EPOCH) PLOT

import sys
import numpy as np
import matplotlib.pyplot as plt
from operator import truediv

loss = []

with open("training.txt" )  as file:
     for line in file:
           # print(line)
            n = line.find("epoch")
            if n != -1:
                continue
            n = line.find(",",6, len(line)-1)
            if n != -1:
                x = line[n+1:n+ 7]
                #print(x)
                #print (type(x))
                x = float(x)
                loss.append(x)
                continue
print(loss)

epoch = []
for i in range(0,100): # 100 epoch number
    epoch.append(float(i+1))
    
print(epoch)

ratio = []
for i in range(0,100): # epoch number
    #r = loss/epoch
    r = map(truediv, loss, epoch)
#    r = round(r,5)
    ratio.append(r)
    
#print(ratio)
p = np.poly1d(np.polyfit(epoch, loss, 10)) # find the right fit please
t = np.linspace(1,100, 2000) # remember to change 100 to epoch number and 2000 scales with epochs as well
plt.plot(epoch, loss, 'o', t, p(t),  '-')
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Loss vs Epoch plot")
plt.show()

In [ ]:
# Evaluate performance on independent sample, un CNN on Test sample
print("Running on full test sample. This may take a moment.")
ret = model.predict(images_test)
np.save("result.pynb",ret[:,1]) #this line doesn't work
print(ret)

In [ ]:
# evaluating ROC curve inputs
from sklearn.metrics import roc_curve
#fpr_keras, tpr_keras, thresholds_keras = roc_curve(keras.utils.to_categorical(df_test["isSharedHit"])[:,1], ret[:,1])
fpr_keras, tpr_keras, thresholds_keras = roc_curve(keras.utils.to_categorical(df_test["nUniqueSimTracksInSharedHit"]>1)[:,1], ret[:,1])

print fpr_keras,tpr_keras
from sklearn.metrics import auc
auc_keras = auc(fpr_keras, tpr_keras)
print auc_keras
print np.isnan(fpr_keras).all()
print len(fpr_keras),len(tpr_keras)
np.save("fpr_keras.npy",fpr_keras)
np.save("tpr_keras.npy",tpr_keras)

In [ ]:
#Plotting ROC curve

fpr_keras = np.load("fpr_keras.npy")
tpr_keras = np.load("tpr_keras.npy")
auc = np.trapz(tpr_keras,fpr_keras)
print auc

plt.figure(1)
plt.plot(fpr_keras, tpr_keras, label='Keras (area = {:.3f})'.format(auc))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve (area = {:.3f})'.format(auc))
plt.savefig("ROC.png")
plt.savefig("ROC.pdf")
plt.show() 


In [ ]:
# Sanity check: no overlap between train and test 
pandas.merge(df_train, df_test, on=[x for x in df_train.columns], how='inner')

In [ ]:
# visualizing our data
print "shared hits with at least 2 sim tracks:"
print float(sum(df_train["nUniqueSimTracksInSharedHit"]>1))/len(df_train["nUniqueSimTracksInSharedHit"])
plt.hist(df_train["nUniqueSimTracksInSharedHit"],histtype="step",bins=6,range=(-0.5,5.5))
plt.xlabel('Hits')
plt.ylabel('Events')
plt.title("Distribution of Hits in Training Data")

In [ ]:
# Visualizing merged (2 or more) vs not merged (1 or 0 (?)) 
################################################################ Look at events with nUniqueSimTracksInSharedHit = 0(seem to be uncategorized ?)
print float(sum(df_train["nUniqueSimTracksInSharedHit"]>1))/len(df_train["nUniqueSimTracksInSharedHit"])
plt.hist(df_train["nUniqueSimTracksInSharedHit"]>1,histtype="step",bins=2,range=(-0.5,1.5))
plt.title("Distribution of Hits in Training Data")
plt.xticks([0,1],("Not Merged","Merged"))
plt.ylabel('Events')
plt.savefig("merged_dist.png")
plt.savefig("merged_dist.pdf")

In [ ]:
# Separating signal and background for train and test data
signal = ret[df_test["nUniqueSimTracksInSharedHit"]>1]
background = ret[df_test["nUniqueSimTracksInSharedHit"]<2]

signal_plt = signal[:,1]
background_plt = background[:,1]

ret_train = model.predict(images_train)

signal_train = ret_train[df_train["nUniqueSimTracksInSharedHit"]>1]
background_train = ret_train[df_train["nUniqueSimTracksInSharedHit"]<2]
signal_train_plt = signal_train[:,1]
background_train_plt = background_train[:,1]

Y_back_hist = np.histogram(background_train_plt, bins = 30, range = (0,1))[0]
X_back_hist = np.histogram(background_train_plt, bins = 30, range = (0,1))[1]

Y_sig_hist = np.histogram(signal_train_plt, bins = 30, range = (0,1))[0]
X_sig_hist = np.histogram(signal_train_plt, bins = 30, range = (0,1))[1]

In [ ]:
#Signal and background plot 
plt.hist(signal_plt, alpha = 0.5, color = 'b', label = 'Signal (test)', range = (0,1), bins = 30)
plt.hist(background_plt, color = 'r', alpha = 0.5, label = 'Background (test)', range = (0,1), bins = 30)
plt.scatter(X_back_hist[0:30] + 0.0166 , Y_back_hist, label='Background (train)', color ='r')
plt.scatter(X_sig_hist[0:30] + 0.0166, Y_sig_hist, label='Signal (train)', color='b')
plt.legend(loc='best')
plt.xlabel('Discriminant')
plt.ylabel('Events')
plt.title('CNN Signal and Background Discriminants')

Other tests

In [ ]:
print ret_train > .46

In [ ]:
print df_train[ret_train[:,1]>.46]

In [ ]:
print df_train[(ret_train[:,1]>.46) & (ret_train[:,1]<.5)]

In [ ]:
# Adding isMergedHit column , not necessary
############# DO NOT RUN THIS !!! #############################

# for train
#merged_hit = df_train["nUniqueSimTracksInSharedHit"]>1
#merged_int = merged_hit.astype(int)
#df_train.insert(0, "isMergedHit", merged_int, True)

# for test
#merged_hit_test = df_test["nUniqueSimTracksInSharedHit"]>1
#merged_int_test = merged_hit_test.astype(int)
#df_test.insert(0, "isMergedHit", merged_int_test, True)

#df_test.head()

In [ ]:
# Adding Discriminant branch to df_test only for now

index = df_test.index
N = len(index)
print N


temp = []
discriminants_test = model.predict(images_test)
for i in range (0,N):
   # print determinant_train[i][0]
    temp.append(discriminants_test[i][1])

#print (temp)
df_test.insert(0, "Discriminants", temp)

#determinant_test = model.predict(images_test)
#print (determinant_test[1][0])
#df_test.insert(0, "Determinant (sig)", determinant_test)
#df_test.insert(0, "Determinant", determinant, True)
#df_test.head()

In [ ]:
print df_test.info()

In [ ]:
# printout
print df_test [(df_test["Discriminants"] > 0.3 ) & (df_test["Discriminants"] < 0.5 ) ] ["Discriminants"]
#df_train.keys, 

In [ ]:
print df_test.columns.get_loc("nUniqueSimTracksInSharedHit")


In [ ]:
# making sure column 21 is "nUniqueSimTracksInSharedHit"
for i in range(0,5000):
    if df_test.iloc[i,22] == 0:
        print df_test.iloc[i,22]

print df_test.info()

In [ ]:
# PIXEL PICTURE SCRIPT 0.3 - 0.5 discriminant events, Looking at df test pixels only for now

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#data = pd.read_hdf('pixelTrain30k.h5')
#data = pd.read_hdf('output_final_copy.h5')
data = df_test [(df_test["Discriminants"] > 0.3 ) & (df_test["Discriminants"] < 0.5 ) ]

hits_column = df_test.columns.get_loc("nUniqueSimTracksInSharedHit")

index = data.index
N = len(index)
print N
#print data.info()

# sorting by discriminant ascending
data = data.sort_values("Discriminants", ascending = True)[0:len(data.index)-1]
# shareds = data[data["nUniqueSimTracksInSharedHit"]==1]
shareds =  data


#shareds.sort_values("Discriminants", ascending = True)
# (ret_train[:,1]>.46) & (ret_train[:,1]<.5) & there
#pixels = shareds[(shareds.columns[6:])].values

pixelColumns = ["pixel_%i" % x for x in range(36)]
pixels = shareds[pixelColumns].values
print(pixels[0].shape)

#print(pixels[0].shape)

index = shareds.index
N = len(index)
print N

# condidtional here if (ret_train[:,1]>.46) & (ret_train[:,1]<.5):
for row,hit,  in enumerate(pixels):
    x_pos = []
    y_pos = []
    charge = []
    for index,pixel in enumerate(hit):
        if pixel!=0:
            x_pos.append(index%6)
            y_pos.append(np.floor(index/6))  
            charge.append(pixel)
    #print(row, ret_train[row][0]) ######################### this should be discriminant number ? ret
    print(row, data.iloc[row,0] )
        
    fig=plt.figure()
    ax=plt.axes()
    cax = plt.hist2d(x_pos,y_pos,weights=charge,bins=(6,6),range=((0,6),(0,6)))
    cb=fig.colorbar(cax[3])
    cb.set_ticks([0,max(charge)])
    cb.set_label("normalized adc",rotation=-90)

    
    # Title, note that 21 refers to the columns position of the column nUniqueSimTracksInSharedHit
    hits_column = df_test.columns.get_loc("nUniqueSimTracksInSharedHit")
    if data.iloc[row,hits_column] == 1 : # 1 hit
        plt.title("Not Merged Cluster Charge Map")
    elif data.iloc[row,hits_column] > 1 : # 2 or more hits
        plt.title("Merged Cluster Charge Map")
    else : # 0 hits
        plt.title("Null Cluster Charge Map")
    
    plt.xlabel("x")
    plt.ylabel("y")
   
    # uncomment below to save plots
    #plt.savefig(str(row)+".png")
    plt.show()

In [ ]:
# Distribution of 0.3 - 0.5 discriminant events
data["Discriminants"].plot(kind='hist', title = "Discriminant in the range [0.3, 0.5]", bins = 100, figsize=(12,6))

In [ ]:
# PIXEL PICTURE SCRIPT 0.9 - 1 discriminant events, Looking at df test pixels only for now

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#data = pd.read_hdf('pixelTrain30k.h5')
#data = pd.read_hdf('output_final_copy.h5')
data = df_test [df_test["Discriminants"] > 0.9 ]

index = data.index
N = len(index)
print N
#print data.info()

# sorting by discriminant ascending
data = data.sort_values("Discriminants", ascending = True)[0:len(data.index)-1]
# shareds = data[data["nUniqueSimTracksInSharedHit"]==1]
shareds =  data


#shareds.sort_values("Discriminants", ascending = True)
# (ret_train[:,1]>.46) & (ret_train[:,1]<.5) & there
#pixels = shareds[(shareds.columns[6:])].values ###### add the 2 lines

pixelColumns = ["pixel_%i" % x for x in range(36)]
pixels = shareds[pixelColumns].values
print(pixels[0].shape)

index = shareds.index
N = len(index)
print N

# condidtional here if (ret_train[:,1]>.46) & (ret_train[:,1]<.5):
for row,hit,  in enumerate(pixels):
    x_pos = []
    y_pos = []
    charge = []
    for index,pixel in enumerate(hit):
        if pixel!=0:
            x_pos.append(index%6)
            y_pos.append(np.floor(index/6))  
            charge.append(pixel)
    #print(row, ret_train[row][0]) 
    print(row, data.iloc[row,0] ) # row refers to event number, 0 refers to column 0, which is the "Discriminants" column
        
    fig=plt.figure()
    ax=plt.axes()
    cax = plt.hist2d(x_pos,y_pos,weights=charge,bins=(6,6),range=((0,6),(0,6)))
    cb=fig.colorbar(cax[3])
    cb.set_ticks([0,max(charge)])
    cb.set_label("normalized adc",rotation=-90)
 
    # Title, note that 22 refers to the columns position of the column "nUniqueSimTracksInSharedHit"
    hits_column = df_test.columns.get_loc("nUniqueSimTracksInSharedHit")
    if data.iloc[row,hits_column] == 1 : # 1 hit
        plt.title("Not Merged Cluster Charge Map")
    elif data.iloc[row,hits_column] > 1 : # 2 or more hits
        plt.title("Merged Cluster Charge Map")
    else : # 0 hits
        plt.title("Null Cluster Charge Map")
    
    plt.xlabel("x")
    plt.ylabel("y")
   
    # uncomment below to save plots
    #plt.savefig(str(row)+".png")
    plt.show()

In [ ]:
# Distribution of 0.9 - 1 discriminant events
data["Discriminants"].plot(kind='hist' , title = "Discriminant in the range [0.9, 1]", bins = 100, figsize=(12,6))

In [ ]:
# PIXEL PICTURE SCRIPT 0 - 0.1 discriminant events, Looking at df test pixels only for now

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#data = pd.read_hdf('pixelTrain30k.h5')
#data = pd.read_hdf('output_final_copy.h5')
data = df_test [(df_test["Discriminants"] < 0.1 )]

index = data.index
N = len(index)
print N
#print data.info()

# sorting by discriminant ascending
data = data.sort_values("Discriminants", ascending = True)[0:len(data.index)-1]
# shareds = data[data["nUniqueSimTracksInSharedHit"]==1]
shareds =  data


shareds.sort_values("Discriminants", ascending = True)
# (ret_train[:,1]>.46) & (ret_train[:,1]<.5) & there
#pixels = shareds[(shareds.columns[6:])].values

pixelColumns = ["pixel_%i" % x for x in range(36)]
pixels = shareds[pixelColumns].values
print(pixels[0].shape)

index = shareds.index
N = len(index)
print N

# condidtional here if (ret_train[:,1]>.46) & (ret_train[:,1]<.5):
for row,hit,  in enumerate(pixels):
    x_pos = []
    y_pos = []
    charge = []
    for index,pixel in enumerate(hit):
        if pixel!=0:
            x_pos.append(index%6)
            y_pos.append(np.floor(index/6))  
            charge.append(pixel)
    #print(row, ret_train[row][0]) ######################### this should be discriminant number ? ret
    print(row, data.iloc[row,0] )
        
    fig=plt.figure()
    ax=plt.axes()
    cax = plt.hist2d(x_pos,y_pos,weights=charge,bins=(6,6),range=((0,6),(0,6)))
    cb=fig.colorbar(cax[3])
    cb.set_ticks([0,max(charge)])
    cb.set_label("normalized adc",rotation=-90)
 
    # Title, note that 22 refers to the columns position of the column nUniqueSimTracksInSharedHit
    hits_column = df_test.columns.get_loc("nUniqueSimTracksInSharedHit")
    if data.iloc[row,hits_column] == 1 : # 1 hit
        plt.title("Not Merged Cluster Charge Map")
    elif data.iloc[row,hits_column] > 1 : # 2 or more hits
        plt.title("Merged Cluster Charge Map")
    else : # 0 hits
        plt.title("Null Cluster Charge Map")
    
    plt.xlabel("x")
    plt.ylabel("y")
   
    # uncomment below to save plots
    #plt.savefig(str(row)+".png")
    plt.show()

In [ ]:
# Distribution of 0 - 0.1 discriminant events
data["Discriminants"].plot(kind='hist', title = "Discriminant in the range [0, 0.1]", bins = 100, figsize=(12,6))

In [ ]:
# WIDTH + HEIGHT SCRIPT

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#data = pd.read_hdf('pixelTrain30k.h5')
#data = pd.read_hdf('output_final_copy.h5')
data = df_test 

index = data.index
N = len(index)
print N
#print data.info()

# sorting by discriminant ascending
data = data.sort_values("Discriminants", ascending = True)[0:len(data.index)-1]
# shareds = data[data["nUniqueSimTracksInSharedHit"]==1]
shareds =  data


#shareds.sort_values("Discriminants", ascending = True)
# (ret_train[:,1]>.46) & (ret_train[:,1]<.5) & there
#pixels = shareds[(shareds.columns[6:])].values

pixelColumns = ["pixel_%i" % x for x in range(36)]
pixels = shareds[pixelColumns].values

print(pixels[0].shape)


index = shareds.index
N = len(index)
print N

width = [] # dx
height = [] # dy
# condidtional here if (ret_train[:,1]>.46) & (ret_train[:,1]<.5):




for row,hit,  in enumerate(pixels):
    x_pos = []
    y_pos = []
    charge = []
    #init the array
    arra = np.zeros((6,6))
    #ro, co = (6, 6) 
    #arra = [[0]*co]*ro 
    #print(arra)
    for index,pixel in enumerate(hit):   
        if pixel!=0:
            x_pos.append(index%6)
            y_pos.append(np.floor(index/6))  
            charge.append(pixel)
            #print (index%6, np.floor(index/6), pixel)
            arra [5 - int(np.floor(index/6))][int(index%6)]= pixel
    
    print arra
    charge_in_x = np.sum(arra,axis=0)
    charge_in_y = np.sum(arra,axis=1)
    #print charge_in_x
    charge_x_values = np.where(charge_in_x>0)[0]
    charge_y_values = np.where(charge_in_y>0)[0]
    #print charge_x_values
    wid = charge_x_values[-1] - charge_x_values[0] + 1
    hei = charge_y_values[-1] - charge_y_values[0] + 1
    width.append(wid)
    height.append(hei)
    #print(row, ret_train[row][0]) ######################### this should be discriminant number ? ret
    print(row, np.around(data.iloc[row,0], decimals= 5), wid, hei )
 




In [ ]:
#removing single pixel 

#pixel_number = np.sum(pixels[np.sum(pixels>0,axis=1)==1], axis = 1)
#print pixel_number

pixel_number = pixels.astype(bool).sum(axis=1)
#print len(pixel_number)
#print len(data)
#data.insert(1, "Pixel_number", pixel_number)
print data.info()
data_new = data[data["Pixel_number"]>1]
print data_new.info()

In [ ]:
# Adding height and width branch
data.insert(1, "Height", height)
data.insert(1, "Width", width)
print data.info()

In [ ]:
# Separating signal and background for train and test data
signal = data[(data["nUniqueSimTracksInSharedHit"]>1)]
background = data[(data["nUniqueSimTracksInSharedHit"]<2)]

signal_plt_width = signal["Width"]
background_plt_width = background["Width"]

#ret_train = model.predict(images_train)

#signal_train = ret_train[df_train["nUniqueSimTracksInSharedHit"]>1]
#background_train = ret_train[df_train["nUniqueSimTracksInSharedHit"]<2]
#signal_train_plt = signal_train[:,1]
#background_train_plt = background_train[:,1]

#Y_back_hist = np.histogram(background_train_plt, bins = 30, range = (0,1))[0]
#X_back_hist = np.histogram(background_train_plt, bins = 30, range = (0,1))[1]

#Y_sig_hist = np.histogram(signal_train_plt, bins = 30, range = (0,1))[0]
#X_sig_hist = np.histogram(signal_train_plt, bins = 30, range = (0,1))[1]

In [ ]:
#Signal and background plot # hist type = ste[]
plt.hist(signal_plt_width, alpha = 0.5, color = 'b', label = 'Signal (test)', range = (1,6), bins = 6)
plt.hist(background_plt_width, color = 'r', alpha = 0.5, label = 'Background (test)', range = (1,6), bins = 6)
#plt.scatter(X_back_hist[0:30] + 0.0166 , Y_back_hist, label='Background (train)', color ='r')
#plt.scatter(X_sig_hist[0:30] + 0.0166, Y_sig_hist, label='Signal (train)', color='b')
plt.legend(loc='best')
plt.title('CNN Signal and Background Width')
plt.xlabel('Width')
plt.ylabel('Events')

In [ ]:
#signal = data[(data["nUniqueSimTracksInSharedHit"]>1)]
#background = data[(data["nUniqueSimTracksInSharedHit"]<2)]

signal_plt_height = signal["Height"]
background_plt_height = background["Height"]

plt.hist(signal_plt_height, alpha = 0.5, color = 'b', label = 'Signal (test)', range = (1,6), bins = 6)
plt.hist(background_plt_height, color = 'r', alpha = 0.5, label = 'Background (test)', range = (1,6), bins = 6)

plt.legend(loc='best')
plt.title('CNN Signal and Background Height')
plt.xlabel('Height')
plt.ylabel('Events')